In [1]:
import numpy as np
import pandas as pd
import glob
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [10]:
path = "/Users/user/Desktop/Oka/ML-data/24-1-25"

In [29]:
df = pd.DataFrame(columns=['x座標','y座標','z座標','V[V]','T_cell[℃]','W_ch[mm]','i[A/m^2]'])

T_range = ["/*_100℃_1.5ms","/*_110℃_1.5ms","/*_120℃_1.5ms","/*_130℃_1.5ms","/*_140℃_1.5ms","/*_150℃_1.5ms"]
for T in T_range:
    W_ch_range = ["_0.4mm","_0.8mm","_1.6mm"] #0.2mmはいったん消す
    for W_ch in W_ch_range:
        files2 = glob.glob(path + T + W_ch)
        for filename in files2:
            df1 = pd.read_csv(filename,names=['x座標','y座標','z座標','i[A/m^2]'], delim_whitespace=True, header=8)
            v = re.findall(r'_3D(.*?)V_',filename)
            df1.insert(3,'V[V]', v[0])
            t = re.findall(r'V_(.*?)℃_',filename)
            df1.insert(4,'T_cell[℃]',t[0])
            w = re.findall(r'ms_(.*?)mm',filename)
            df1.insert(5,'W_ch[mm]',w[0])
            df1 = df1.sample(30000)
            df = pd.concat([df, df1])
df['V[V]'] = df['V[V]'].astype(float)
df['T_cell[℃]'] = df['T_cell[℃]'].astype(float)
df['W_ch[mm]'] = df['W_ch[mm]'].astype(float)
df.reset_index(drop=True, inplace=True)
df

C:\Users\user\AppData\Local\Temp\ipykernel_20572\141963562.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1])


,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
0,0.000572,0.004392,0.000560,0.45,100.0,0.4,16994.714636
1,0.000492,0.001969,0.000550,0.45,100.0,0.4,16746.199513
2,0.000316,0.003349,0.000617,0.45,100.0,0.4,16717.660839
3,0.000324,0.009653,0.000559,0.45,100.0,0.4,16233.411241
4,0.000564,0.007913,0.000555,0.45,100.0,0.4,16726.998598
...,...,...,...,...,...,...,...
5939995,0.001030,0.006226,0.000270,0.90,150.0,1.6,1370.487861
5939996,0.001690,0.000985,0.000314,0.90,150.0,1.6,1954.007729
5939997,0.000950,0.001443,0.000350,0.90,150.0,1.6,1453.974841
5939998,0.001030,0.001576,0.000250,0.90,150.0,1.6,1430.834264


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940000 entries, 0 to 5939999
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   x座標        float64
 1   y座標        float64
 2   z座標        float64
 3   V[V]       float64
 4   T_cell[℃]  float64
 5   W_ch[mm]   float64
 6   i[A/m^2]   float64
dtypes: float64(7)
memory usage: 317.2 MB


In [14]:
df.describe()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
count,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06
mean,6.665804e-04,5.319920e-03,4.333333e-04,6.500000e-01,1.250000e+02,9.333333e-01,8.386806e+03
std,4.805481e-04,3.109630e-03,1.292462e-04,1.581139e-01,1.707825e+01,4.988877e-01,6.842299e+03
min,4.000000e-06,0.000000e+00,2.500000e-04,4.000000e-01,1.000000e+02,4.000000e-01,-8.680095e-02
25%,2.920000e-04,2.626010e-03,3.251667e-04,5.000000e-01,1.100000e+02,4.000000e-01,2.567537e+03
50%,5.800000e-04,5.319000e-03,4.001667e-04,6.500000e-01,1.250000e+02,8.000000e-01,6.523222e+03
75%,9.300000e-04,8.015930e-03,5.748333e-04,8.000000e-01,1.400000e+02,1.600000e+00,1.297754e+04
max,1.990000e-03,1.063800e-02,6.500000e-04,9.000000e-01,1.500000e+02,1.600000e+00,2.899154e+04


In [30]:
#V=0.65,T=110,W=1.6の値だけ外れていたので補完する
# 条件に基づいて 'i[A/m^2]' のデータを抽出し、その抽出したデータに対して平均を取る
subset1 = df.loc[(df['V[V]'] == 0.6) & (df['T_cell[℃]'] == 110) & (df['W_ch[mm]'] == 1.6), 'i[A/m^2]']
subset2 = df.loc[(df['V[V]'] == 0.7) & (df['T_cell[℃]'] == 110) & (df['W_ch[mm]'] == 1.6), 'i[A/m^2]']

# 平均を計算する
df_re = (subset1.values + subset2.values) / 2
df.loc[1770000:1799999,'i[A/m^2]'] = df_re
df.loc[1770000:1799999,'x座標'] = df.loc[1740000:1769999,'x座標']
df.loc[1770000:1799999,'y座標'] = df.loc[1740000:1769999,'y座標']
df.loc[1770000:1799999,'z座標'] = df.loc[1740000:1769999,'z座標']#ランダムサンプリングされているから座標を合わせるのは無理

In [20]:
df_re

array([5650.2939635 , 4792.05788495, 5496.4602487 , ..., 4333.37793796,
       3534.32193505, 5844.32657626])

In [16]:
df.to_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V")

In [17]:
for col in df.columns:
    scaler =  StandardScaler()
    df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1, 1))
    
df.head()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
0,-0.013693,0.615798,1.347558,-1.264911,-1.46385,-1.069045,1.269524
1,-0.013693,-0.823866,1.676388,-1.264911,-1.46385,-1.069045,1.314717
2,0.002954,-1.593271,1.676388,-1.264911,-1.46385,-1.069045,1.349897
3,-0.579714,0.949978,1.676388,-1.264911,-1.46385,-1.069045,1.157429
4,-1.079144,0.633220,1.265028,-1.264911,-1.46385,-1.069045,1.266472


In [18]:
df.describe()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
count,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06
mean,-2.519867e-16,-6.421010e-16,-1.903585e-16,2.773882e-15,2.001960e-17,-2.705364e-15,1.674106e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.378801e+00,-1.710789e+00,-1.418481e+00,-1.581139e+00,-1.463850e+00,-1.069045e+00,-1.230726e+00
25%,-7.794858e-01,-8.663119e-01,-8.369040e-01,-9.486833e-01,-8.783101e-01,-1.069045e+00,-8.479027e-01
50%,-1.801701e-01,-2.956983e-04,-2.566160e-01,2.808667e-15,0.000000e+00,-2.672612e-01,-2.763920e-01
75%,5.481650e-01,8.669875e-01,1.094810e+00,9.486833e-01,8.783101e-01,1.336306e+00,6.693600e-01
max,2.753980e+00,1.710197e+00,1.676388e+00,1.581139e+00,1.463850e+00,1.336306e+00,3.017174e+00


In [19]:
df.to_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V_std")